In [2]:
import tensorflow as tf
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import cv2

import os
import random
# Use multiple models at the same time with the same eps for untargeted and targeted
# Try out with different labels for the targeted

2025-05-13 12:33:54.287786: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747132434.315831   30515 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747132434.324026   30515 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-05-13 12:33:54.350479: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
pretrained_model = tf.keras.applications.MobileNetV2(include_top=True,
                                                     weights='imagenet')
pretrained_model.trainable = False

# ImageNet labels
decode_predictions = tf.keras.applications.mobilenet_v2.decode_predictions

2025-05-12 13:12:43.084237: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


In [3]:
# Helper function to preprocess the image so that it can be inputted in MobileNetV2
def preprocess(image):
  image = tf.cast(image, tf.float32)
  image = tf.image.resize(image, (224, 224))
  image = tf.keras.applications.mobilenet_v2.preprocess_input(image)
  image = image[None, ...]
  return image

# Helper function to extract labels from probability vector
def get_imagenet_label(probs):
  return decode_predictions(probs, top=1)[0][0]

def display_images(image, description):
  _, label, confidence = get_imagenet_label(pretrained_model.predict(image))
  plt.figure()
  plt.imshow(image[0]*0.5+0.5)
  plt.title('{} \n {} : {:.2f}% Confidence'.format(description,
                                                   label, confidence*100))
  plt.show()

In [4]:
# For clipping the input vector
def clip_delta(t, eps):
    return tf.clip_by_value(t, -eps, eps)


# Generates targeted adversarial images
def generate_adv_pertubation(model, optimizer, loss, img, delta, label, target_label, eps, max_iterations=500):
    # delta = tf.Variable(tf.zeros_like(img), trainable=True)
    
    for step in range(max_iterations):
        with tf.GradientTape() as tape:
            tape.watch(delta)
            adv = preprocess(img + delta)

            preds = model(adv,training=False)
            # print(preds)
            originalLoss = -loss(tf.convert_to_tensor([label]), preds)
            targetLoss = loss(tf.convert_to_tensor([target_label]), preds)
            totalLoss = originalLoss + targetLoss

            # if step % 50 == 0:
            #     print("step: {}, loss: {}...".format(step, totalLoss.numpy()), "Range:", tf.reduce_min(adv).numpy(), "to", tf.reduce_max(adv).numpy())

            gradients = tape.gradient(totalLoss, delta)
            optimizer.apply_gradients([(gradients, delta)])
            delta.assign_add(clip_delta(delta, eps))
    
    return delta


In [5]:
def save_image(tensor, target_folder, filename):

    # Remove batch dimension if present
    if len(tensor.shape) == 4:
        tensor = tensor[0]

    # Convert from [-1, 1] to [0, 255]
    tensor = (tensor + 1.0) / 2.0  # [-1,1] → [0,1]
    tensor = tf.clip_by_value(tensor, 0.0, 1.0)
    tensor = tf.image.convert_image_dtype(tensor, dtype=tf.uint8)

    # Encode as JPEG
    encoded = tf.io.encode_jpeg(tensor)

    # Ensure the directory exists
    os.makedirs(target_folder, exist_ok=True)

    # Save to file
    path = os.path.join(target_folder, filename)
    tf.io.write_file(path, encoded)
    print(f"Image saved to: {path}")


def load_and_preprocess_image(path):
    raw = tf.io.read_file(path)
    img = tf.image.decode_image(raw, channels=3)
    return tf.cast(img, tf.float32)

In [6]:
def generate_adversarial_images(images, source_folder, target_folder, label, target_label, eps, iterations):
    i, mx = 0, len(images)
    for img_name in images:
        print(f"{i}/{mx}: {img_name}")
        image_path = tf.keras.utils.get_file(img_name, 'file://' + source_folder + img_name)
        image_raw = tf.io.read_file(image_path)
        image = tf.image.decode_image(image_raw)
        
        loss = tf.keras.losses.SparseCategoricalCrossentropy()
        adam = tf.keras.optimizers.Adam(1e-2)
        
        baseImage = tf.constant(image / 1, dtype=tf.float32)
        delta = tf.Variable(tf.zeros_like(baseImage), trainable=True)

        target_pertubation = generate_adv_pertubation(pretrained_model, adam, loss, baseImage, delta, label, target_label, eps, iterations)

        adv = (baseImage + target_pertubation)
        adv = preprocess(adv)

        _, cl, conf = get_imagenet_label(pretrained_model.predict(adv))
        # display_images(adv, descriptions[i])
        print(f"Classified as {cl} with {conf}% confidence.")
        
        # if cl == "pineapple":
        save_image(adv, target_folder, img_name)
        i += 1

In [ ]:
full_path = "./banana_images/data/train/banana/"
target_path = "./adversarial_images/"
image_files = [f for f in os.listdir(full_path) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]

sub_images = image_files[:100]

eps = 0.01
label = tf.Variable(954)
target_label = tf.Variable(953)
iterations = 100

generate_adversarial_images(sub_images, full_path, target_path, label, target_label, eps, iterations)

2025-04-30 17:23:05.589357: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: INVALID_ARGUMENT: Shapes of all inputs must match: values[0].shape = [333,500,3] != values[1].shape = [460,640,3]


InvalidArgumentError: {{function_node __wrapped__Pack_N_100_device_/job:localhost/replica:0/task:0/device:CPU:0}} Shapes of all inputs must match: values[0].shape = [333,500,3] != values[1].shape = [460,640,3] [Op:Pack] name: stack

## Untargeted attacks

In [ ]:
def create_adversarial_pattern(input_image, input_label, loss_object, model):
  with tf.GradientTape() as tape:
    tape.watch(input_image)
    prediction = model(input_image)
    loss = loss_object(input_label, prediction)

  # Get the gradients of the loss w.r.t to the input image.
  gradient = tape.gradient(loss, input_image)
  # Get the sign of the gradients to create the perturbation
  signed_grad = tf.sign(gradient)
  return signed_grad

In [15]:
def generate_untargeted_adversarial_images(images, source_folder, target_folder, label, eps):
    i, mx = 0, len(images)
    label = tf.one_hot(label, 1000)
    label = tf.reshape(label, (1, 1000))
    for img_name in images:
        print(f"{i}/{mx}: {img_name}")

        image_path = tf.keras.utils.get_file(img_name, 'file://' + source_folder + img_name)
        image_raw = tf.io.read_file(image_path)
        image = tf.image.decode_image(image_raw)

        image_pr = preprocess(image)
        
        loss_object = tf.keras.losses.CategoricalCrossentropy()

        pertubation = create_adversarial_pattern(image_pr, label, loss_object, pretrained_model)

        adv = image_pr + eps * pertubation
        adv = tf.clip_by_value(adv, -1, 1)

        _, cl, conf = get_imagenet_label(pretrained_model.predict(adv))
        # display_images(adv, descriptions[i])
        print(f"Classified as {cl} with {conf}% confidence.")

        save_image(adv, target_folder, img_name)
        i += 1

In [ ]:
from tensorflow.keras.applications import ResNet50, VGG16, InceptionV3, EfficientNetB0, MobileNetV3Small, MobileNetV2
from tensorflow.keras.applications.resnet50 import preprocess_input as res_pre, decode_predictions as res_decode
from tensorflow.keras.applications.vgg16 import preprocess_input as vgg_pre, decode_predictions as vgg_decode
from tensorflow.keras.applications.inception_v3 import preprocess_input as inc_pre, decode_predictions as inc_decode
from tensorflow.keras.applications.mobilenet_v3 import preprocess_input as mn_pre, decode_predictions as mn_decode
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input as mn2_pre, decode_predictions as mn2_decode

model_names = ("ResNet50", "VGG", "MobileNetV3Small")
models = (MobileNetV2(weights='imagenet'), ResNet50(weights='imagenet'), VGG16(weights='imagenet'), MobileNetV3Small(weights='imagenet'))
pre_functions = (res_pre, vgg_pre, inc_pre, mn_pre)
decode_functions = (res_decode, vgg_decode, inc_decode, mn_decode)

/home/paul/anaconda3/envs/nlp/lib/python3.12/site-packages/keras/src/applications/mobilenet_v3.py:452: UserWarning: `input_shape` is undefined or non-square, or `rows` is not 224. Weights for input shape (224, 224) will be loaded as the default.
  return MobileNetV3(


In [ ]:
def generate_untargeted_adversarial_images_multiple_models(images, source_folder, target_folder, label, eps, models, decoders, preprocessors):
    i, mx = 0, len(images)
    label = tf.one_hot(label, 1000)
    label = tf.reshape(label, (1, 1000))
    for img_name in images:
        print(f"{i}/{mx}: {img_name}")

        image_path = tf.keras.utils.get_file(img_name, 'file://' + source_folder + img_name)
        image_raw = tf.io.read_file(image_path)
        image = tf.image.decode_image(image_raw)
        image_pr = preprocess(image)
        for j in range(len(models)):
            
            loss_object = tf.keras.losses.CategoricalCrossentropy()

            pertubation = create_adversarial_pattern(image_pr, label, loss_object, models[j])
            # plt.imshow(pertubation[0] * 0.5 + 0.5)
            display_images(pertubation, "")

            adv = image_pr + eps * pertubation
            adv = tf.clip_by_value(adv, -1, 1)
            image_pr = adv
        print(f"Dimensions: raw = {image_raw.shape}, img = {image.shape}, preprocessed = {image_pr.shape}, adv = {adv.shape}")
        _, cl, conf = get_imagenet_label(pretrained_model.predict(adv))
        # display_images(adv,"")
        print(f"Classified as {cl} with {conf}% confidence.")

        save_image(adv, target_folder, img_name)
        i += 1

In [3]:
full_path = "./banana_images/data/train/banana/"
target_path = "./adversarial_images_untargeted/"
image_files = [f for f in os.listdir(full_path) if f.lower().endswith(('.png', '.jpg', '.jpeg'))][:2]

sub_images = image_files

eps = 0.01
label = tf.Variable(954)

generate_untargeted_adversarial_images_multiple_models(sub_images, full_path, target_path, label, eps, models, decode_functions, pre_functions)

2025-05-13 12:35:23.621731: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


NameError: name 'generate_untargeted_adversarial_images_multiple_models' is not defined